# Займемся поиском start/end токенов ответа в вопросе

In [1]:
from langchain.document_loaders import HuggingFaceDatasetLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
# from transformers import AutoTokenizer, AutoModelForQuestionAnswering
# from transformers import AutoTokenizer, pipeline
# from langchain import HuggingFacePipeline
# from langchain.chains import RetrievalQA
# import torch

/home/user/work/tinkoff/.venv/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/user/work/tinkoff/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_name = "kuznetsoffandrey/sberquad"
page_content_column = "context"

loader = HuggingFaceDatasetLoader(dataset_name, page_content_column)

data = loader.load()

data[:1]

[Document(metadata={'id': 62310, 'title': 'SberChallenge', 'question': 'чем представлены органические остатки?', 'answers': {'text': ['известковыми выделениями сине-зелёных водорослей'], 'answer_start': [109]}}, page_content='"\\u0412 \\u043f\\u0440\\u043e\\u0442\\u0435\\u0440\\u043e\\u0437\\u043e\\u0439\\u0441\\u043a\\u0438\\u0445 \\u043e\\u0442\\u043b\\u043e\\u0436\\u0435\\u043d\\u0438\\u044f\\u0445 \\u043e\\u0440\\u0433\\u0430\\u043d\\u0438\\u0447\\u0435\\u0441\\u043a\\u0438\\u0435 \\u043e\\u0441\\u0442\\u0430\\u0442\\u043a\\u0438 \\u0432\\u0441\\u0442\\u0440\\u0435\\u0447\\u0430\\u044e\\u0442\\u0441\\u044f \\u043d\\u0430\\u043c\\u043d\\u043e\\u0433\\u043e \\u0447\\u0430\\u0449\\u0435, \\u0447\\u0435\\u043c \\u0432 \\u0430\\u0440\\u0445\\u0435\\u0439\\u0441\\u043a\\u0438\\u0445. \\u041e\\u043d\\u0438 \\u043f\\u0440\\u0435\\u0434\\u0441\\u0442\\u0430\\u0432\\u043b\\u0435\\u043d\\u044b \\u0438\\u0437\\u0432\\u0435\\u0441\\u0442\\u043a\\u043e\\u0432\\u044b\\u043c\\u0438 \\u0432\\u044b\

In [3]:
# Что-то с кодировкой, поэтому исправляем
for document in data:
    document.page_content = document.page_content.encode().decode('unicode_escape')

In [4]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

# docs = text_splitter.split_documents(data)

In [5]:
# Берем какую-нибудь bert для расчёта эмбединга, этот справляется вроде бы нелпохо
modelPath = "sergeyzh/LaBSE-ru-turbo"

model_kwargs = {'device':'cpu'}

encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/tmp/ipykernel_885217/2651821630.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [6]:
text = "Проверка"
query_result = embeddings.embed_query(text)
query_result[:3]

[0.0207399632781744, 0.019484564661979675, -0.06904035806655884]

In [18]:
db = FAISS.from_documents(data, embeddings)

In [19]:
db.save_local("faiss_index")

In [20]:
db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

In [21]:
question = "чем представлены органические остатки?"
searchDocs = db.similarity_search(question)
print(searchDocs[0].page_content)

"В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий."


In [22]:
retriever = db.as_retriever()

In [23]:
docs = retriever.invoke("чем представлены органические остатки?")
print(docs[0].page_content)

"В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий."


In [24]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

# https://www.kaggle.com/code/tttrrraaahhh/rubert-sberquad/notebook
model_name = "./ru-bert-finetuned-squad"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Функция для QA
def answer_question(question, context):
    inputs = tokenizer(question, context, return_tensors="pt")
    outputs = model(**inputs)
    
    answer_start = outputs.start_logits.argmax()
    answer_end = outputs.end_logits.argmax() + 1
    
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
    return answer

In [25]:
question = "что относится к числу древнейших растительных остатков?"
docs = retriever.invoke(question)

print(answer_question(question, docs[0].page_content))

скопления графито - углистого вещества


In [ ]:
# pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16)

# question = ""


# docs = retriever.get_relevant_documents(question)

# retrived_text = '\n\n'.join([doc.page_content for doc in docs])
# messages = [
#     {
#         "role": "system",
#         "content": f'''Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
#         context: {retrived_text}'''
#     },
#     {"role": "user", "content": f'{question}'}
# ]
# prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
# outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
# print(outputs[0]["generated_text"][len(prompt):])
# question = input()
# # <|system|>

In [ ]:
# question = "Who is Thomas Jefferson?"
# result = qa.invoke({"query": question})
# print(result["result"])